In [1]:
import pandas as pd
import math
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(style='whitegrid')

In [2]:
data = pd.read_csv('data/data_train_DF.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

# Replacing clim1 and clim2 by wind
data['wind'] = np.sqrt(data.clim1**2+data.clim2**2)
data.drop(['clim1','clim2'], axis=1, inplace=True)

# Relative humidity
data['RH'] = 100 * np.exp(17.625*(data.clim3-273.15)/(data.clim3-39.11)) / np.exp(17.625*(data.clim4-273.15)/(data.clim4-39.11))

# Dropping NA in new dataframe
data_validation = data[data.BA.isna() | data.CNT.isna()]
data.dropna(inplace=True)
    
data

,CNT,BA,lon,lat,area,year,month,lc1,lc2,lc3,...,clim3,clim4,clim5,clim6,clim7,clim8,clim9,clim10,wind,RH
0,0.0,0.000000,-95.25,49.25,0.24,1993,3,0.000006,0.015857,0.000023,...,265.457680,268.867126,-0.005898,9.187450e+06,-5231370.50,97849.906250,-0.000340,0.000448,0.277765,76.304978
1,0.0,0.000000,-94.75,49.25,0.39,1993,3,0.000005,0.002749,0.000002,...,265.521764,268.412354,-0.001131,6.993830e+06,-4851900.00,97954.703125,-0.000264,0.000462,0.232549,79.479839
2,0.0,0.000000,-122.75,48.75,0.48,1993,3,0.002420,0.103964,0.003870,...,276.699820,280.594666,-0.010519,1.052566e+07,-4860741.00,100808.468750,-0.001945,0.004545,1.281072,75.576717
3,3.0,8.000000,-122.25,48.75,1.00,1993,3,0.002988,0.237442,0.004040,...,274.943327,278.574371,-0.008420,9.359787e+06,-4653411.50,98474.648438,-0.001256,0.006174,0.963919,76.703795
4,0.0,0.000000,-121.75,48.75,1.00,1993,3,0.000000,0.004782,0.000196,...,271.235317,274.578064,-0.005976,7.479946e+06,-3889238.75,91660.625000,-0.000502,0.008110,0.540732,77.703374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563978,6.0,134.600006,-80.75,25.75,1.00,2015,9,0.014934,0.017630,0.001066,...,296.832658,300.201447,-0.006076,1.632964e+07,-3884615.75,101323.140625,-0.004034,0.006132,0.490861,81.341554
563979,1.0,30.000000,-80.25,25.75,0.66,2015,9,0.014798,0.014336,0.001253,...,297.139517,300.414673,-0.005535,1.552289e+07,-3794893.75,101300.398438,-0.003913,0.006199,0.765370,81.842714
563980,0.0,0.000000,-81.25,25.25,0.28,2015,9,0.000000,0.000000,0.000000,...,297.105902,301.003082,-0.007343,1.738194e+07,-3929039.00,101333.078125,-0.004233,0.003420,1.085526,78.826490
563981,2.0,179.100006,-80.75,25.25,0.76,2015,9,0.015875,0.039337,0.000373,...,297.059054,300.905426,-0.006938,1.732889e+07,-4057722.00,101332.671875,-0.004125,0.003929,1.024579,79.061076


In [3]:
# List of labels by type
clim_labels = ['altiMean', 'altiSD', 'clim3', 'clim4', 'clim5', 'clim6', 'clim7', 'clim8','clim9', 'clim10', 'wind', 'RH']
land_labels = ['lc1','lc2','lc3','lc4','lc5','lc6','lc7','lc8','lc9','lc10','lc11','lc12','lc13','lc14','lc15','lc16','lc17','lc18']

In [4]:
from sklearn.preprocessing import StandardScaler

# Splitting data to train/test
X_train = data[data.year.isin([1995,2005,2015])].drop('BA', axis=1)
y_train = data[data.year.isin([1995,2005,2015])][['BA']]
X_test  = data[data.year.isin([1997,2007])].drop('BA', axis=1)
y_test  = data[data.year.isin([1997,2007])][['BA']]

# Standardizing climate variables
scaler = StandardScaler()
X_train[clim_labels] = scaler.fit_transform(X_train[clim_labels])
X_test[clim_labels] = scaler.fit_transform(X_test[clim_labels])

In [5]:
# Adding mean CNT per voxel
mean_CNT_voxel = X_train.groupby(['lon','lat'])[['CNT']].mean()
X_train['mean_fire'] = X_train.apply(lambda x: mean_CNT_voxel.loc[x.lon,x.lat], axis=1)
X_test['mean_fire'] = X_test.apply(lambda x: mean_CNT_voxel.loc[x.lon,x.lat], axis=1)

In [6]:
# Defining Thresholds and Scoring Function
u_CNT = list(range(0,11))+[2*x for x in range(6,16)]+[10*x for x in range(4,11)]
u_BA  = [0,1]+[10*x for x in range(1,11)]+[50*x for x in range(3,7)]+[400,500,1000,1500,2000,5000]+[10000*x for x in range(1,6)]+[100000]

w_CNT = [1-(1+(x+1)**2/1000)**(-1/4) for x in u_CNT] 
w_CNT = [x/w_CNT[-1] for x in w_CNT] 

w_BA  = [1-(1+(x+1)/1000)**(-1/4) for x in u_BA]
w_BA  = [x/w_BA[-1] for x in w_BA]

def cnt_score(y_test, prediction_dist):  
    S_CNT  = 0
    for i in range(len(actual)):
        S_CNT = S_CNT + np.sum([w_CNT[x]*(int(u_CNT[x]>=y_test.iloc[i])-prediction_dist.iloc[i,x])**2 for x in range(len(u_CNT))])
    return S_CNT

def ba_score(y_test, prediction_dist):  
    S_BA  = 0
    for i in range(len(y_test)):
        S_BA = S_BA + np.sum([w_BA[x]*(int(u_BA[x]>=y_test.iloc[i])-prediction_dist.iloc[i,x])**2 for x in range(len(u_BA))])
    return S_BA

In [7]:
# Appoximation of the prediction distribution - log1p
def dist_log1p(prediction):
    ba_distribution = pd.DataFrame(data=0, index=np.arange(prediction.size), columns=u_BA)
    ba_distribution = ba_distribution.apply(lambda x: stats.norm.cdf(np.log1p(x.name), loc=prediction[x.index], scale=np.std(prediction)), result_type='expand')
    return ba_distribution

# Models

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import Lasso, BayesianRidge

### Random Forest Regression

In [9]:
%%time

mRF = RandomForestRegressor(max_depth=5, n_estimators=500, n_jobs=8, bootstrap=False, random_state=42)
mRF.fit(X_train.drop('CNT', axis=1), np.ravel(np.log1p(y_train)))

prediction = mRF.predict(X_test.drop('CNT', axis=1))
ba_distribution = dist_log1p(prediction)
ba_distribution.to_csv('prediction_ba.csv', index=False)

## SCORE=4280

CPU times: user 5min 29s, sys: 375 ms, total: 5min 29s
Wall time: 43.1 s


### Lasso Enhanced Random Forest Regression

In [10]:
%%time

mLasso = Lasso(alpha=0.1)
mLasso.fit(X_train.drop('CNT', axis=1), np.ravel(np.log1p(y_train)))

residuals = np.ravel(np.log1p(y_train))-mLasso.predict(X_train.drop('CNT', axis=1))

mRERF = RandomForestRegressor(max_depth=5, n_estimators=500, n_jobs=10, bootstrap=False, random_state=42)
mRERF.fit(X_train.drop('CNT', axis=1), residuals)

prediction = mLasso.predict(X_test.drop('CNT', axis=1)) + mRERF.predict(X_test.drop('CNT', axis=1))
ba_distribution = dist_log1p(prediction)
ba_distribution.to_csv('prediction_ba.csv', index=False)

## SCORE=4345

CPU times: user 6min 45s, sys: 488 ms, total: 6min 45s
Wall time: 42.3 s


### Random Forest Classifier - with/without CNT

In [11]:
# New y_train that represents intervals of BA
new_y_train = pd.Categorical(pd.cut(np.ravel(y_train), bins=[-1]+u_BA, right=True, labels=u_BA)).add_categories(-1).fillna(-1)
new_y_train = pd.DataFrame(new_y_train, columns=['threshold'], index=y_train.index)

In [ ]:
# Maybe useful later

param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 500, 1000, 1500, 2000]
}

search_grid = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=8)
search_grid.fit(X_train, new_y_train.threshold)

search_grid.best_params_

In [12]:
%%time

mRFC = RandomForestClassifier(max_depth=5, n_estimators=1000, n_jobs=8, bootstrap=False, max_features=None, random_state=42)
mRFC.fit(X_train.drop('CNT', axis=1), new_y_train.threshold)

ba_distribution = pd.DataFrame(mRFC.predict_proba(X_test.drop('CNT', axis=1)), columns=new_y_train.threshold.unique().categories).cumsum(axis=1)
ba_distribution.drop(-1, axis=1).to_csv('prediction_ba.csv', index=False)

## SCORE=4080

CPU times: user 15min 2s, sys: 954 ms, total: 15min 3s
Wall time: 1min 56s


In [13]:
%%time

mRFC = RandomForestClassifier(max_depth=5, n_estimators=1000, n_jobs=8, bootstrap=False, max_features=None, random_state=42)
mRFC.fit(X_train, new_y_train.threshold)

ba_distribution = pd.DataFrame(mRFC.predict_proba(X_test), columns=new_y_train.threshold.unique().categories).cumsum(axis=1)
ba_distribution.drop(-1, axis=1).to_csv('prediction_ba.csv', index=False)

## SCORE=3755

CPU times: user 9min, sys: 669 ms, total: 9min 1s
Wall time: 1min 10s


In [14]:
# What the output look like :
ba_distribution

,0,1,10,20,30,40,50,60,70,80,...,1500,2000,5000,10000,20000,30000,40000,50000,100000,-1
0,0.0,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0
1,0.0,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0
2,0.0,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0
3,0.0,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0
4,0.0,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49037,0.0,0.0,0.105109,0.555912,0.70073,0.770803,0.815474,0.848175,0.872117,0.886715,...,0.984234,0.988905,0.991533,0.997372,0.999416,1.0,1.0,1.0,1.0,1.0
49038,0.0,0.0,0.105109,0.555912,0.70073,0.770803,0.815474,0.848175,0.872117,0.886715,...,0.984234,0.988905,0.991533,0.997372,0.999416,1.0,1.0,1.0,1.0,1.0
49039,0.0,1.0,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0
49040,0.0,0.0,0.105109,0.555912,0.70073,0.770803,0.815474,0.848175,0.872117,0.886715,...,0.984234,0.988905,0.991533,0.997372,0.999416,1.0,1.0,1.0,1.0,1.0


### Pipeline - Grid Search CV

In [ ]:
param_grid = {'alpha': np.logspace(0,99,num=10)}

mLasso = Lasso()

clf = GridSearchCV(mLasso, param_grid, n_jobs=5, cv=5)
clf.fit(X_train.drop('CNT', axis=1), np.ravel(np.log1p(y_train)))
print(clf.best_params_)

prediction = clf.predict(X_test.drop('CNT', axis=1))
ba_distribution = dist_log1p(prediction)
ba_distribution.to_csv('prediction_ba.csv', index=False)

######

pca = PCA()
rForest = RandomForestRegressor(bootstrap=False, random_state=42)

pipe = Pipeline(steps=[('pca', pca), ('rF', rForest)])

param_grid = {
    'pca__n_components': [5, 10, 15, 30, 40],
    'rF__max_depth': [1,5],
    'rF__n_estimators': [500,1000]
}

search = GridSearchCV(pipe, param_grid, n_jobs=8)
search.fit(X_train.drop('CNT', axis=1), np.ravel(np.log1p(y_train)))
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

prediction = search.predict(X_test.drop('CNT', axis=1))
ba_distribution = dist_log1p(prediction)
ba_distribution.to_csv('prediction_ba.csv', index=False)

In [15]:
%%time

clf = BayesianRidge(n_iter=500)
clf.fit(X_train.drop('CNT', axis=1), np.log1p(y_train.BA))

prediction = clf.predict(X_test.drop('CNT', axis=1))
ba_distribution = pd.DataFrame(data=0, index=np.arange(prediction.size), columns=u_BA)
ba_distribution = ba_distribution.apply(lambda x: stats.norm.cdf(np.log1p(x.name), loc=prediction[x.index], scale=clf.lambda_**-1), 
                                        result_type='expand')
ba_distribution.to_csv('prediction_ba.csv', index=False)
ba_distribution

## SCORE=4224

CPU times: user 3.88 s, sys: 114 ms, total: 3.99 s
Wall time: 2.04 s


,0,1,10,20,30,40,50,60,70,80,...,1000,1500,2000,5000,10000,20000,30000,40000,50000,100000
0,0.453390,0.621479,0.912806,0.960463,0.977017,0.984910,0.989333,0.992071,0.993887,0.995154,...,0.999982,0.999994,0.999997,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0
1,0.551266,0.710665,0.945665,0.977360,0.987509,0.992104,0.994582,0.996071,0.997034,0.997691,...,0.999994,0.999998,0.999999,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0
2,0.608676,0.758766,0.960045,0.984185,0.991545,0.994777,0.996480,0.997485,0.998126,0.998557,...,0.999997,0.999999,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0
3,0.532329,0.694131,0.940201,0.974668,0.985880,0.991008,0.993794,0.995478,0.996572,0.997323,...,0.999993,0.999998,0.999999,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0
4,0.487282,0.653425,0.925554,0.967211,0.981280,0.987869,0.991512,0.993744,0.995212,0.996228,...,0.999988,0.999996,0.999998,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49037,0.088319,0.177559,0.549406,0.699168,0.776860,0.824758,0.857225,0.880621,0.898224,0.911903,...,0.998131,0.999180,0.999559,0.999950,0.999992,0.999999,1.0,1.0,1.0,1.0
49038,0.227696,0.374488,0.766963,0.870071,0.914086,0.938021,0.952802,0.962692,0.969687,0.974843,...,0.999771,0.999913,0.999958,0.999997,1.000000,1.000000,1.0,1.0,1.0,1.0
49039,0.405981,0.574787,0.892039,0.949006,0.969598,0.979668,0.985416,0.989029,0.991455,0.993166,...,0.999970,0.999990,0.999995,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0
49040,0.221943,0.367237,0.761052,0.865969,0.911035,0.935641,0.950881,0.961102,0.968346,0.973694,...,0.999754,0.999906,0.999954,0.999996,1.000000,1.000000,1.0,1.0,1.0,1.0
